In [1]:
!pip install wandb
!wandb login


     |████████████████████████████████| 2.1MB 12.5MB/s 
     |████████████████████████████████| 133kB 48.1MB/s 
     |████████████████████████████████| 163kB 47.4MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=f6bf0d409d19703cda9e033d548b1da51e7c8b0ec9e019297238993bbff359ac
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=bef45ee8a81aabec7dee02a9385e57193076d49966cef0ea187313f0350a66f1
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Ap

In [2]:
import pickle
from sklearn.model_selection import train_test_split
from spacy.lang.en import English

import random
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from pathlib import Path
import time

import wandb



In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/NLP/project/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/NLP/project


In [ ]:
# %mkdir NYTData
# !wget https://www.dropbox.com/sh/xu9tu5hmjhuddwk/AAD31tK6oEoGlhpRZzeu3Y3Ya/NYTcorpus_train.p.gz?dl=1 --directory-prefix=NYTData
# !gunzip -c NYTData/NYTcorpus_train.p.gz\?dl\=1 > NYTData/NYTcorpus_train.p

In [ ]:
# %mkdir vector_cache
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip --directory-prefix=vector_cache
# !unzip vector_cache/wiki-news-300d-1M.vec.zip -d vector_cache/

In [12]:
PAD = '<PAD>'
PAD_ID = 0
UNK = '<UNK>'
UNK_ID = 1
VOCAB_PREFIX = [PAD, UNK]

VEC_PATH = Path('vector_cache') / 'wiki-news-300d-1M.vec'
MAX_VOCAB = 25000

batch_size = 64
validation_split = .3
shuffle_dataset = True
random_seed = 42

In [13]:
class BaseVocab:
    def __init__(self, data, lower=False):
        self.data = data
        self.lower = lower
        self.build_vocab()
        
    def normalize_unit(self, unit):
        if self.lower:
            return unit.lower()
        else:
            return unit
        
    def unit2id(self, unit):
        unit = self.normalize_unit(unit)
        if unit in self._unit2id:
            return self._unit2id[unit]
        else:
            return self._unit2id[UNK]
    
    def id2unit(self, id):
        return self._id2unit[id]
    
    def map(self, units):
        return [self.unit2id(unit) for unit in units]
        
    def build_vocab(self):
        NotImplementedError()
        
    def __len__(self):
        return len(self._unit2id)

In [14]:
class PretrainedWordVocab(BaseVocab):
    def build_vocab(self):
        self._id2unit = VOCAB_PREFIX + self.data
        self._unit2id = {w:i for i, w in enumerate(self._id2unit)}

In [15]:
class LabelVocab(BaseVocab):
    def build_vocab(self):
        self._id2unit = self.data
        self._unit2id = {w:i for i, w in enumerate(self._id2unit)}

In [16]:
class Pretrain:
    def __init__(self, vec_filename, max_vocab=-1):
        self._vec_filename = vec_filename
        self._max_vocab = max_vocab
        
    @property
    def vocab(self):
        if not hasattr(self, '_vocab'):
            self._vocab, self._emb = self.read()
        return self._vocab
    
    @property
    def emb(self):
        if not hasattr(self, '_emb'):
            self._vocab, self._emb = self.read()
        return self._emb
        
    def read(self):
        if self._vec_filename is None:
            raise Exception("Vector file is not provided.")
        print(f"Reading pretrained vectors from {self._vec_filename}...")
        
        words, emb, failed = self.read_from_file(self._vec_filename, open_func=open)
        
        if failed > 0: # recover failure
            emb = emb[:-failed]
        if len(emb) - len(VOCAB_PREFIX) != len(words):
            raise Exception("Loaded number of vectors does not match number of words.")
            
        # Use a fixed vocab size
        if self._max_vocab > len(VOCAB_PREFIX) and self._max_vocab < len(words):
            words = words[:self._max_vocab - len(VOCAB_PREFIX)]
            emb = emb[:self._max_vocab]
                
        vocab = PretrainedWordVocab(words, lower=True)
        print("Done Reading")
        
        return vocab, emb
        
    def read_from_file(self, filename, open_func=open):
        """
        Open a vector file using the provided function and read from it.
        """
        first = True
        words = []
        failed = 0
        with open_func(filename, 'rb') as f:
            for i, line in enumerate(f):
                try:
                    line = line.decode()
                except UnicodeDecodeError:
                    failed += 1
                    continue
                if first:
                    # the first line contains the number of word vectors and the dimensionality
                    first = False
                    line = line.strip().split(' ')
                    rows, cols = [int(x) for x in line]
                    emb = np.zeros((rows + len(VOCAB_PREFIX), cols), dtype=np.float32)
                    continue

                line = line.rstrip().split(' ')
                emb[i+len(VOCAB_PREFIX)-1-failed, :] = [float(x) for x in line[-cols:]]
                words.append(' '.join(line[:-cols]))
        return words, emb, failed

In [17]:

class NYTDataSet(Dataset):
    def __init__(self, vectorized_data):
        self.data = vectorized_data
  
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]


In [18]:
def pad_sequences(batch):
    max_len = max([len(x[0]) for x in batch])
    padded_sequences = torch.zeros((len(batch), max_len), dtype=torch.long)
    labels = torch.zeros((len(batch), len(batch[0][1])), dtype=torch.float)
    for i, sample in enumerate(batch):

      padded_sequences[i, :len(sample[0])] = sample[0]
      labels[i, :] = sample[1]

    padded_sequences = padded_sequences.to(device)
    labels = labels.to(device)
    
    return padded_sequences, labels

In [19]:
pretrain = Pretrain(VEC_PATH, MAX_VOCAB)

In [20]:

nlp = English()
tokenizer = nlp.tokenizer

# Check if we are running on a CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [21]:
%%time 
vectorized_train_data = torch.load('vectorized_10000_train.pt')
train_data = NYTDataSet(vectorized_data=vectorized_train_data)

CPU times: user 667 ms, sys: 295 ms, total: 962 ms
Wall time: 2.43 s


In [22]:
n_training_samples=len(train_data)
n_training_samples

10000

In [24]:
%%time
vectorized_test_data = torch.load('vectorized_10000_test.pt')
test_data = NYTDataSet(vectorized_data=vectorized_test_data)
print(len(test_data))

10000
CPU times: user 735 ms, sys: 255 ms, total: 991 ms
Wall time: 1.15 s


In [25]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [26]:
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, collate_fn=pad_sequences)
validation_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, collate_fn=pad_sequences)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=pad_sequences)

In [27]:
class CNN(nn.Module):
    def __init__(self, pretrain, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding.from_pretrained(
            torch.from_numpy(pretrain.emb), 
            padding_idx=pad_idx, 
            freeze=True
        )
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):           
        #text = [batch size, sent len]

        embedded = self.embedding(text)     
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)  
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]    
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]     
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [28]:
INPUT_DIM = len(pretrain.vocab)
EMBEDDING_DIM = pretrain.emb.shape[1]
N_FILTERS = 100
FILTER_SIZES = [3,4,5,6]
OUTPUT_DIM = 538
DROPOUT = 0.6

model = CNN(pretrain, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_ID)

Reading pretrained vectors from vector_cache/wiki-news-300d-1M.vec...
Done Reading


In [29]:
optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

run = wandb.init(
    entity='ut-mit-news-classify',
    project="NYT Multilabeling",
)
# Magic
wandb.watch(model)

wandb: Currently logged in as: kristjan (use `wandb login --relogin` to force relogin)


In [30]:
def multi_label_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    confusion_vector = rounded_preds / y

    true_positives = torch.sum(confusion_vector==1)
    false_positives = torch.sum(torch.isinf(confusion_vector)) 
    false_negatives = torch.sum(confusion_vector==0)
    true_negatives = torch.sum(torch.isnan(confusion_vector)) 

    accuracy = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f_score = (2 * precision * recall) / (precision + recall)
    
    return accuracy, precision, recall, f_score

In [31]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f_score = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch[0]).squeeze(1)
        
        loss = criterion(predictions, batch[1])
        
        acc, precision, recall, f_score = multi_label_accuracy(predictions, batch[1])
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_precision += precision.item()
        epoch_recall += recall.item()
        epoch_f_score += f_score.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), \
        epoch_precision / len(iterator), epoch_recall / len(iterator), \
        epoch_f_score / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f_score = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch[0]).squeeze(1)
            
            loss = criterion(predictions, batch[1])
            
            acc, precision, recall, f_score = multi_label_accuracy(predictions, batch[1])

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_precision += precision.item()
            epoch_recall += recall.item()
            epoch_f_score += f_score.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), \
        epoch_precision / len(iterator), epoch_recall / len(iterator), \
        epoch_f_score / len(iterator)

In [33]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
%%time
patience = 7
epochs_of_no_improvement = 0
best_valid_loss = float('inf')

wandb.config.early_stopping_patience = patience
wandb.config.training_samples=n_training_samples

model_file_name = f'nyt_cnn_classifier_trained_with_{n_training_samples}_samples.pt'

epoch = 0

while True:

    start_time = time.time()
    
    train_loss, train_acc, train_precision, train_recall, train_f_score \
        = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc, valid_precision, valid_recall, valid_f_score \
        = evaluate(model, validation_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        print(f'New validation loss {valid_loss} is better than the best validation loss {best_valid_loss} so far.')
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_file_name)
        epochs_of_no_improvement = 0
    else: 
        epochs_of_no_improvement += 1
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | ' +
          f'Train Precision: {train_precision*100:.2f}% | Train Recall: {train_recall*100:.2f}% | ' +
          f'Train F1-score: {train_f_score*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | ' +
          f'Val. Precision: {valid_precision*100:.2f}% | Val. Recall: {valid_recall*100:.2f}% | ' +
          f'Val. F1-score: {valid_f_score*100:.2f}%')
    
    wandb.log({"train_loss": train_loss, 
                "train_precision": train_precision, 
                "train_f_score": train_f_score, 
                "train_acc": train_acc,
                "train_recall": train_recall,
               "valid_loss": valid_loss,
               "valid_acc": valid_acc,
               "valid_precision": valid_precision,
               "valid_recall": valid_recall,
               "valid_f_score": valid_f_score
                })
    # check if the training should be stopped and then stop the training
    if epochs_of_no_improvement == patience : 
        print(f'Early stopping, on epoch: {epoch+1}.')
        break
    epoch += 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [ ]:
n_training_samples

In [ ]:
model = CNN(pretrain, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_ID)
model.load_state_dict(torch.load(model_file_name))
model = model.to(device)

In [ ]:
start_time = time.time()
test_loss, test_acc, test_precision, test_recall, test_f_score \
    = evaluate(model, test_loader, criterion)
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)

print(f'Epoch: test | Epoch Time: {epoch_mins}m {epoch_secs}s')
print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | ' +
      f'Test Precision: {test_precision*100:.2f}% | Test Recall: {test_recall*100:.2f}% | ' +
      f'Test F1-score: {test_f_score*100:.2f}%')

In [ ]:
wandb.log({"test_acc": test_acc,
            "test_precision": test_precision,
            "test_recall": test_recall,
            "test_f_score": test_f_score
            })
run.finish()